In [1]:
import torch
import numpy as np
from torch import nn
import pandas as pd
import cv2
NUM_DATA = 4999#

In [2]:
#increased red range a bit
#removed cropping
#changed blurring kernel size
def image_preprocess(img):
    img = cv2.GaussianBlur(img, (5, 5), cv2.BORDER_DEFAULT)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Lower and upper HSV bounds for red (at both ends of the spectrum)
    lowerb1 = np.array([0, 70, 20])
    upperb1 = np.array([10, 255, 255])
    lowerb2 = np.array([160, 70, 20])
    upperb2 = np.array([180, 255, 255])
    # Masking the path to be followed
    lower_mask  = cv2.inRange(img_hsv, lowerb1, upperb1)
    upper_mask  = cv2.inRange(img_hsv, lowerb2, upperb2)

    mask = lower_mask + upper_mask
    img_edges = cv2.Canny(mask, 50, 200, None, 3)#[mask.shape[0]//2:]
    img_edges = cv2.resize(img_edges, (640, 240))
    # last part is done for normalization
#     img_edges = img_edges/255.0
    return img_edges

In [3]:
data = pd.read_csv('data/data.csv')
loc = data.location.values
NUM_DATA = len(data)
Y = data.values[:NUM_DATA, 1:]
Y = np.array(Y, dtype=np.float32)
Y = Y[:,0]
# Y = torch.from_numpy(np.float32(Y)).to(device)
img_size = (240, 640) #that is after cropping it in the preprocess
#reshaping Y to be onehotencoded  -1->pos0, 0->pos1, 1->pos2
tmp = np.zeros((Y.shape[0],3))
indices = np.array(Y+1,dtype = np.int16)
tmp[np.arange(indices.shape[0]),indices] = 1
Y = torch.tensor(tmp, dtype=torch.float32)

In [20]:
X = []

for i in loc:
    try:
        im = cv2.imread('images/' + i+'.png',cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (640,240))
        X.append(im)
    except:
        print('f')
        pass
    
    if len(X) == NUM_DATA:
        break
X = np.array(X)



In [21]:
print(X)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [22]:
X = torch.tensor(np.expand_dims(X,axis=1))

In [23]:
X = X.float()

In [24]:
print(X.shape)
print(Y.shape)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.Size([5000, 1, 240, 640])
torch.Size([5000, 3])


In [25]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1 ,24, (5,5), (2,2))
        self.conv2 = nn.Conv2d(24 ,36, (5,5), (2,2))
        self.conv3 = nn.Conv2d(36 ,48, (5,5), (2,2))
        self.conv4 = nn.Conv2d(48 ,64, (5,5),)
        self.conv5 = nn.Conv2d(64 ,64, (5,5),)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(83904, 100)
        self.linear2 = nn.Linear(100, 50)
        self.linear3 = nn.Linear(50, 10)
        self.linear4 = nn.Linear(10, 3)

    def forward(self,x):
        out = torch.relu(self.conv1(x))
        out = torch.relu(self.conv2(out))
        out = torch.relu(self.conv3(out))
        out = torch.relu(self.conv4(out))
        out = torch.relu(self.conv5(out))
        out = torch.relu(self.linear1(self.flatten(out)))
        out = torch.relu(self.linear2(out))
        out = torch.relu(self.linear3(out))
        out = torch.relu(self.linear4(out))
        return out

model = Model().to(device)

In [26]:
X.shape

torch.Size([5000, 1, 240, 640])

In [27]:
model(X[0:2]).shape

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [28]:
class CarData(torch.utils.data.Dataset):
    def __init__(self,transform = None):
        self.X = X
        self.Y = Y
        self.n_samples = self.X.shape[0]
        self.transform = transform
    def __getitem__(self,index):
        sample = (self.X[index], self.Y[index])
        if self.transform:
            sample = self.transform(sample)
        return sample
    def __len__(self):
        return self.n_samples

In [29]:
batch_size = 32
dataset = CarData()
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True)

In [33]:
print(device)
crit = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(params=model.parameters(), lr = 0.001)
epochs = 20
n_total_steps = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
#         print(i)
        images = images.to(device)
        labels = labels.to(device)
        #forward pass
        outputs = model(images)
        loss = crit(outputs, labels)
        #backward pass
        optim.zero_grad()
        loss.backward()
        optim.step()
        if (i+1) % 100 ==0:
            print(f'Epoch [{epoch+1}/{epochs}], step [{i+1}/{n_total_steps},loss: {loss.item()}]')
print("Finished Training")

cuda
Epoch [1/20], step [100/157,loss: 0.08301929384469986]
Epoch [2/20], step [100/157,loss: 0.003588191932067275]
Epoch [3/20], step [100/157,loss: 0.002160094678401947]
Epoch [4/20], step [100/157,loss: 0.020034201443195343]
Epoch [5/20], step [100/157,loss: 0.002392152789980173]
Epoch [6/20], step [100/157,loss: 0.004576648585498333]
Epoch [7/20], step [100/157,loss: 0.0022724270820617676]
Epoch [8/20], step [100/157,loss: 0.0021807400044053793]
Epoch [9/20], step [100/157,loss: 0.0031755096279084682]
Epoch [10/20], step [100/157,loss: 0.011748252436518669]
Epoch [11/20], step [100/157,loss: 7.372086201939965e-06]
Epoch [12/20], step [100/157,loss: 0.0014284044736996293]
Epoch [13/20], step [100/157,loss: 0.0011442765826359391]
Epoch [14/20], step [100/157,loss: 0.006448961794376373]
Epoch [15/20], step [100/157,loss: 0.0010218432871624827]
Epoch [16/20], step [100/157,loss: 0.01829943247139454]
Epoch [17/20], step [100/157,loss: 0.00432192999869585]
Epoch [18/20], step [100/157,lo

In [34]:
# torch.save(model, 'torch_model.pt')

In [35]:
model = model.to('cpu')
torch.save(model, 'torch_model2_cpu.pt')

In [47]:
# list(model.parameters())

[Parameter containing:
 tensor([[[[ 1.2694e-01, -1.2638e-02, -1.2346e-01, -1.4248e-01, -1.4573e-01],
           [ 1.6437e-01,  8.3088e-02,  1.5177e-01, -1.1167e-02,  6.1372e-02],
           [-1.5687e-01,  1.7451e-01, -1.5103e-01, -9.5684e-02, -1.9553e-01],
           [ 1.8797e-01,  1.1418e-01, -6.7225e-03, -1.1609e-01, -3.6593e-02],
           [-1.1732e-01,  1.9405e-01, -1.4721e-01, -4.9920e-04,  5.2418e-02]]],
 
 
         [[[ 6.1746e-02,  2.2832e-02, -6.0863e-02, -1.5810e-01, -1.5365e-01],
           [-6.0727e-02,  3.6959e-02, -1.9058e-01, -1.1323e-01, -1.5906e-01],
           [ 1.5021e-01, -1.3452e-01,  1.1667e-01, -1.1703e-01, -1.6935e-01],
           [-9.5860e-02, -1.5689e-01,  1.1760e-01, -8.6015e-02, -1.3958e-02],
           [ 1.0802e-01,  4.6854e-02,  1.4731e-01, -6.8988e-02,  1.3333e-01]]],
 
 
         [[[ 4.3424e-02,  1.5561e-01,  7.6944e-02,  5.5652e-02, -1.0355e-01],
           [-6.3114e-02, -1.2417e-01, -1.3473e-01, -1.9182e-01, -1.3348e-01],
           [ 1.8070e-01,  5.3